In [1]:
import pandas as pd
import re
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import string
from sklearn.cluster import KMeans
from collections import defaultdict
from kiwipiepy import Kiwi

## 토크나이저 정의

In [2]:
tokenizer = AutoTokenizer.from_pretrained('monologg/koelectra-small-v3-discriminator')
# tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

## 리뷰 불러오기 & 전처리

In [3]:
reviews = pd.read_csv("./review_떡볶이_ver2.1.csv")['review_content']

reviews = [re.sub(r"<[^>]+>\s+(?=<)|<[^>]+>", "", text) for text in reviews]
reviews = [re.sub(r"[^가-힣a-zA-Z0-9\n\s]", "", text) for text in reviews]

reviews = pd.read_csv("./review_떡볶이_ver2_pre.csv")['review_content']
reviews = [re.sub(r"<[^>]+>\s+(?=<)|<[^>]+>", "", text) for text in reviews]
reviews = [re.sub(r"[^가-힣a-zA-Z0-9\n\s]", "", text).strip() for text in reviews]

In [4]:
def filter_corpus(corpus):
    corpus = [text for text in corpus if text.find("배송") == -1]
    corpus = [text for text in corpus if text.find("도움") == -1]
    corpus = [text for text in corpus if len(text) > 10]
    corpus = [text for text in corpus if len(text) < 200]
    corpus = [text for text in corpus if text.find("년") == -1]
    corpus = [text for text in corpus if text.find("ml") == -1]
    corpus = [text for text in corpus if text.find("날짜") == -1]
    corpus = [text for text in corpus if text.find("22") == -1]
    corpus = [text for text in corpus if text.find("23") == -1]
    corpus = [text for text in corpus if text.find("기한") == -1]
    corpus = [text for text in corpus if text.find("미리") == -1]
    corpus = [text for text in corpus if text.find("감사") == -1]
    return corpus

## 문장 단위로 분리하기

In [5]:
kiwi = Kiwi()

split_texts = []
for review in reviews:
    sents = kiwi.split_into_sents(review)
    sents = [sent.text for sent in sents]
    split_texts.append(sents)

corpus = []
for idx, text in enumerate(split_texts):
    for t in text:
        corpus.append(str(idx) + " " + t)

corpus = filter_corpus(corpus)

## 클러스터링 진행하기

In [6]:
text_id = 1
clusters = ["평가", "조리"]
# data = clusters + filter_corpus(split_texts[text_id])
data = clusters + corpus
n = len(clusters)

# Vectorizer
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
tokenizer_func = lambda x: tokenizer.tokenize(x.translate(remove_punct_dict))
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

# Feature vectorize
feature_vect = vectorizer.fit_transform(data)

# 임의 클러스터로 클러스터 진행
km_cluster = KMeans(n_clusters=n, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect[:n])

# 임의 클러스터로 새로운 클러스터링 초기화
kmeans_new = KMeans(init=km_cluster.cluster_centers_, n_clusters=n)
kmeans_new.fit(feature_vect[n:])

result = [[] for _ in range(n)]
for idx, label in enumerate(km_cluster.labels_):
    result[label].append(clusters[idx])

for idx, label in enumerate(kmeans_new.labels_):
    result[label].append(data[idx+n])

result_dict = {}
for r in result:
    r_dict = defaultdict(list)
    for text in r[1:]:
        index = text.split(" ")[0]
        t =" ".join(text.split(" ")[1:])
        r_dict[index].append(t)
    # result_dict[r[0]] = r[1:]
    result_dict[r[0]] = dict(r_dict)

/opt/homebrew/Caskroom/miniforge/base/envs/torch-env2/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1307: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)


## 클러스터별 + 리뷰별로 분리하기

In [7]:
filtered_text = [[] for _ in range(len(split_texts))]

for idx in result_dict["평가"]:
    filtered_text[int(idx)].extend(result_dict["평가"][idx])

text_list = [ ". ".join(text) for text in filtered_text ]

## 상품별로 리뷰 분리하기

In [8]:
filtered_reviews = []
for i in range(10):
    filtered_reviews.append(" ".join(text_list[i * 20:i * 20 + 20]))

In [9]:
filtered_reviews

['간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요. 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요. 보통같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요. 냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다 맛 가성비 재구매의사 냉동 떡볶이중 최고입니다. 떡복이 땡길때 바로 해먹을수 있어서 좋더라구요. 다진소고기 볶다가 물부어 떡볶이 만들면 최고에요. 라면사리까지 함께 끓이면 세식구 한끼식사로도 가능하더라구요. 그나마 식사대용이라고 위안삼으며 한끼 맛있게 먹죠. 비주얼이 좀더 먹음직스러워지거든요. 깜장소스는 단맛조절이 가능해요 아주 맛있게 먹었던 기억이 있는 제품이라 또 구매 했습니다. 이 제품은 매워서 막둥이가 안먹네요. 볶아서 조리하면 더욱더 맛이 좋다기에 이번에도그렇게 만들었어요. 역시 맛있게 아이와 잘 먹었습니다 오랜만에 구입한 추억의떡볶이 여전히 매콤 달콤하고 맛있어요. 국물이 연해지긴 했지만 원래도 진한 국물스타일이라 많이 싱겁지않고 맛있었어요. 예전에 정말 자주 구입해서 먹곤했는데스믈스믈 가격이 많이 올랐지요 대체떡볶이로 다른거 많이 사다먹었는데 역시 원조라 그런지 가장 맛있는 떡볶이중에 하나인 것 같아요. 여기에 들어있는 어묵도 저는 개인적으로 좋아요 두껍지 않아 양념도 잘 베고 맛있거든요 안녕하세요 v떡볶이를 사랑하는 결이네 삼인방가족 입니다. 주말에 가족과함께 떡볶이를 먹기위해 서치하던중 저번에 먹은적 있는추억의 국민학교 떡볶이 오리지널 을 발견 바로 겟하였습니당. 로켓프레쉬로 할인해서 저렴하게 득템하였네요. 바로 주말에 봉지 뜯어서 해먹어 보았습니다. 맛도 옛날 초딩앞 떡볶이 맛 살짝 나서 추억돋습니다. 떡뽁이 구매 알아보시는분들에게 강추천드립니다. 유 감쟈합니당 v 추억의 국민학교 떡볶이는 제 최애 음식 중 다섯 손가락 안에 듭니다. 일단 맛이 있습니다. 오리지널 맛과 매운맛이 있는데 오리지널 맛은 남녀노소 먹기 부담

## 상품별 리뷰 Document 길이 확인하기

In [10]:
print(f"Orinigal text length: {[sum([len(k) for k in reviews][i * 20: i * 20 + 20]) for i in range(10)]}")
print(f"Filtered text lengths: {[len(k) for k in filtered_reviews]}")

Orinigal text length: [7366, 7990, 6896, 6428, 6115, 6531, 3495, 7172, 4653, 6752]
Filtered text lengths: [3801, 5081, 4328, 4934, 4716, 5205, 2644, 5786, 3418, 4132]
